In [1]:
import requests
import pandas as pd
import xarray as xr
import sqlite3
from sqlalchemy import create_engine 
import os

In [6]:
departement = ["45"]
for dpt in departement:
    if len(dpt) < 10 :
        requete_dpt += '0' + str(dpt) + '%2C'
    else :
        requete_dpt += str(dpt) + '%2C'
requete_dpt = requete_dpt[0:-3]    

url = f"https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/stations?code_departement={requete_dpt}&format=json&size=10000"
print(url)
get= requests.get(f"{url}")

https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/stations?code_departement=45045045&format=json&size=10000


# Récupération des données

## Requête API et création du dataframe

In [249]:
def f_list_piezzo(departement:list, nbre_mesures:int=50) -> list:
    """
    departement : liste des départements recherchés
    date : date au format Y-m-d (2022-05-19)
    return : liste des code_bss et autres informations
    """
    requete_dpt = ""
    for dpt in departement:
        if dpt < 10 :
            requete_dpt += '0' + str(dpt) + '%2C'
        else :
            requete_dpt += str(dpt) + '%2C'
    requete_dpt = requete_dpt[0:-3]    
        
    url = f"https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/stations?code_departement={requete_dpt}&format=json&nb_mesures_piezo_min={nbre_mesures}&size=10000"
    print(url)
    get= requests.get(f"{url}")
    data_piezo = pd.DataFrame(columns=['code_bss',
                                       'date_debut',
                                      'date_fin',
                                       'nb_mesures_piezo',
                                       'code_dpt',
                                      'code_commune_insee',
                                      'nom_commune',
                                      'latitude',
                                      'longitude',
                                      'altitude_station',
                                      'profondeur_investigation',
                                      "codes_bdlisa",
                                      "libelle_pe",
                                      "codes_masse_eau_edl",
                                      "noms_masse_eau_edl"])
    data_piezo["code_bss"] = [x['code_bss'].replace("/","_") for x in get.json()['data']]
    data_piezo["date_debut"] = [pd.to_datetime(x['date_debut_mesure'], format="%Y/%m/%d") for x in get.json()['data']]
    data_piezo["date_fin"] = [pd.to_datetime(x['date_fin_mesure'], format="%Y/%m/%d") for x in get.json()['data']]
    data_piezo["nb_mesures_piezo"] = [x['nb_mesures_piezo'] for x in get.json()['data']]    
    data_piezo["code_dpt"] = [x['code_departement'] for x in get.json()['data']]    
    data_piezo["code_commune_insee"] = [x['code_commune_insee'] for x in get.json()['data']]
    data_piezo["nom_commune"] = [x['nom_commune'] for x in get.json()['data']]
    data_piezo["latitude"] = [x['y'] for x in get.json()['data']]
    data_piezo["longitude"] = [x['x'] for x in get.json()['data']]
    data_piezo["altitude_station"] = [float(x['altitude_station']) for x in get.json()['data']]
    data_piezo["profondeur_investigation"] = [x['profondeur_investigation'] for x in get.json()['data']] 
    
    # Gestion du format tableau et des Nan pour les colonnes suivantes :
    data_piezo["codes_bdlisa"] = [x['codes_bdlisa'] for x in get.json()['data']]
    bdlisa = []
    for item in list(data_piezo["codes_bdlisa"]):
        if item == None:
            bdlisa.append(None)
        else :
            bdlisa.append(item[0])
    data_piezo["codes_bdlisa"] = np.array(bdlisa)
    
    data_piezo["libelle_pe"] = [x['libelle_pe'] for x in get.json()['data']]
    libelle_pe = []
    for item in list(data_piezo["libelle_pe"]):
        if item == None:
            libelle_pe.append(None)
        else :
            libelle_pe.append(item[0])
    data_piezo["libelle_pe"] = np.array(libelle_pe)
    
    data_piezo["codes_masse_eau_edl"] = [x['codes_masse_eau_edl'] for x in get.json()['data']]
    codes_masse_eau_edl = []
    for item in list(data_piezo["codes_masse_eau_edl"]):
        if item == None:
            codes_masse_eau_edl.append(None)
        else :
            codes_masse_eau_edl.append(item[0])
    data_piezo["codes_masse_eau_edl"] = np.array(codes_masse_eau_edl)
    
    
    data_piezo["noms_masse_eau_edl"] = [x['noms_masse_eau_edl'] for x in get.json()['data']]
    noms_masse_eau_edl = []
    for item in list(data_piezo["noms_masse_eau_edl"]):
        if item == None:
            noms_masse_eau_edl.append(None)
        else :
            noms_masse_eau_edl.append(item[0])
    data_piezo["noms_masse_eau_edl"] = np.array(noms_masse_eau_edl)
    
    return data_piezo


In [250]:
liste = [_ for _ in range(1,96)]
df_piezo = f_list_piezzo(liste, nbre_mesures=50).set_index('code_bss')
df_piezo.head()

https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/stations?code_departement=01%2C02%2C03%2C04%2C05%2C06%2C07%2C08%2C09%2C10%2C11%2C12%2C13%2C14%2C15%2C16%2C17%2C18%2C19%2C20%2C21%2C22%2C23%2C24%2C25%2C26%2C27%2C28%2C29%2C30%2C31%2C32%2C33%2C34%2C35%2C36%2C37%2C38%2C39%2C40%2C41%2C42%2C43%2C44%2C45%2C46%2C47%2C48%2C49%2C50%2C51%2C52%2C53%2C54%2C55%2C56%2C57%2C58%2C59%2C60%2C61%2C62%2C63%2C64%2C65%2C66%2C67%2C68%2C69%2C70%2C71%2C72%2C73%2C74%2C75%2C76%2C77%2C78%2C79%2C80%2C81%2C82%2C83%2C84%2C85%2C86%2C87%2C88%2C89%2C90%2C91%2C92%2C93%2C94%2C95&format=json&nb_mesures_piezo_min=50&size=10000


,date_debut,date_fin,nb_mesures_piezo,code_dpt,code_commune_insee,nom_commune,latitude,longitude,altitude_station,profondeur_investigation,codes_bdlisa,libelle_pe,codes_masse_eau_edl,noms_masse_eau_edl
code_bss,,,,,,,,,,,,,,
08272X0745_PZ-,1994-01-21,2004-06-21,125,33,33039,Bègles,44.812534,-0.564909,7.47,6.0,None,None,None,None
04454X0089_PK11.0,1966-11-15,1973-12-27,70,68,68163,Kembs,47.699312,7.507685,232.12,NaN,221AA17,None,CG101,"Nappe d’Alsace, Pliocène de Haguenau et Oligocène"
04134X0057_PK23.0,1968-03-22,1973-12-12,52,68,68253,Ottmarsheim,47.797436,7.530655,216.84,NaN,221AA17,None,CG101,"Nappe d’Alsace, Pliocène de Haguenau et Oligocène"
04454X0064_PK3.0,1966-11-15,1973-12-27,65,68,68286,Rosenau,47.634317,7.546158,245.98,NaN,221AA17,None,CG101,"Nappe d’Alsace, Pliocène de Haguenau et Oligocène"
04138X0071_PK14.0,1966-11-15,1973-12-27,58,68,68254,Petit-Landau,47.721099,7.530516,232.23,NaN,221AA17,None,CG101,"Nappe d’Alsace, Pliocène de Haguenau et Oligocène"


## Recherche d'un piezo en particulier

In [252]:
print(df_piezo.loc[df_piezo.index=="04994X0528_FPZ","noms_masse_eau_edl"])

code_bss
04994X0528_FPZ    Alluvions nappe de Dijon sud (superficielle et...
Name: noms_masse_eau_edl, dtype: object


# Création de la base de données

## Création directe à partir du dataframe avec sqlalchemy

In [253]:
df_piezo.columns

Index(['date_debut', 'date_fin', 'nb_mesures_piezo', 'code_dpt',
       'code_commune_insee', 'nom_commune', 'latitude', 'longitude',
       'altitude_station', 'profondeur_investigation', 'codes_bdlisa',
       'libelle_pe', 'codes_masse_eau_edl', 'noms_masse_eau_edl'],
      dtype='object')

In [254]:
# Création de la base
engine = create_engine('sqlite:///D:\Documents\daranlot\Travail\Analyse Piezo\data\liste_piezos.db',  
                   echo = False, encoding='utf8')

if not os.path.isfile("./data/liste_piezos.db"):
    try : 
        engine.execute("""
                        CREATE TABLE IF NOT EXISTS data_piezo (
                        code_bss TEXT PRIMARY KEY ON CONFLICT IGNORE,
                        date_debut DATE,
                        date_fin DATE,
                        nb_mesures_piezo MEDIUMINT,
                        code_dpt VARCHAR(3),
                        code_commune_insee VARCHAR (6),
                        nom_commune VARCHAR(50),
                        latitude FLOAT,
                        longitude FLOAT,
                        altitude_station FLOAT,
                        profondeur_investigation FLOAT,
                        codes_bdlisa VARCHAR(50),
                        libelle_pe VARCHAR (250),
                        codes_masse_eau_edl VARCHAR (20),
                        noms_masse_eau_edl TEXT,           
                        fichier_piezo TINYINT DEFAULT 0,
                        fichier_era5 TINYINT DEFAULT 0,
                        fichier_combined TINYINT DEFAULT 0,
                        commentaire TEXT
                        )
                        """)
        print("La base de données liste_piezos a été créée.")
    except InterfaceError as e :
        print("Problème durant la création : ", e)
else :
    print("La base de données liste_piezos existe déjà.")

La base de données liste_piezos existe déjà.


In [255]:
df_piezo.to_sql('data_piezo', 
                con = engine,
                if_exists='append')

nb_lignes = engine.execute("SELECT COUNT (*) FROM data_piezo").fetchall()[0][0]
print(f"La base de données contient {nb_lignes} lignes.")
engine.dispose()

La base de données contient 7908 lignes.


## Jointure avec les données BDLISA

## Fichiers disponibles pour les piezos ?

In [15]:
def requete_sql (requete) :
    try:
        connexion = sqlite3.connect('./data/liste_piezos.db')
        curseur = connexion.cursor()
        print("Connexion réussie à SQLite")
        curseur.execute(requete)
        connexion.commit()
        resultat = curseur.fetchall()
        curseur.close()
        connexion.close()
        print("Connexion SQLite est fermée")
        return resultat
    except sqlite3.Error as error:
        print("Erreur lors du mis à jour dans la table", error)

In [257]:
bss_piezo = pd.Series([_.replace(".csv","") for _ in os.listdir(r"./data/piezo") if _.endswith(r".csv")], name="piezo")    
bss_era5 = pd.Series([_.replace(".pkl","") for _ in os.listdir(r"./data/era5") if _.endswith(r".pkl")], name="era5")
bss_combined = np.array([_[5:-4] for _ in os.listdir(r"./data/combined_data/") if _.endswith(r".csv")])
    
requete_fichier_piezo = f"""
        UPDATE data_piezo
        SET fichier_piezo = 1
        WHERE code_bss IN {tuple(bss_piezo)}
         """
requete_sql(requete_fichier_piezo)

requete_fichier_era5 = f"""
        UPDATE data_piezo
        SET fichier_era5 = 1
        WHERE code_bss IN {tuple(bss_era5)}
         """
requete_sql(requete_fichier_era5)

requete_fichier_combined = f"""
        UPDATE data_piezo
        SET fichier_combined = 1
        WHERE code_bss IN {tuple(bss_combined)}
         """
requete_sql(requete_fichier_combined)

Connexion réussie à SQLite
Connexion SQLite est fermée
Connexion réussie à SQLite
Connexion SQLite est fermée
Connexion réussie à SQLite
Connexion SQLite est fermée


[]

In [258]:
requete = f"""
        SELECT COUNT(*)
        FROM data_piezo
        WHERE fichier_combined IS 1
        """

requete_sql(requete)

Connexion réussie à SQLite
Connexion SQLite est fermée


[(96,)]

# Table Départements et régions

In [10]:
df = pd.read_csv("./data/DPT_REG.csv", sep=";", index_col="departement")

In [11]:
df.head()

,code_region
departement,
1,ARA
2,HDF
3,ARA
4,PAC
5,PAC


In [21]:
connexion = sqlite3.connect('./data/liste_piezos.db')
df.to_sql("code_region", connexion, if_exists='replace', index=True)

95